In [12]:
import pandas as pd
import re
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams
#from google.colab import drive
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import warnings
warnings.filterwarnings('ignore')

In [19]:
# df_hlt_train = pd.read_csv("/data/parush/wtwt/healthcare_train.txt", sep='\t')
# df_hlt_test = pd.read_csv("/data/parush/wtwt/healthcare_test.txt", sep='\t')
# df_ent_train = pd.read_csv("/data/parush/wtwt/entertainment_train.txt", sep='\t')
# df_ent_test = pd.read_csv("/data/parush/wtwt/entertainment_test.txt", sep='\t')
# print("Length of Health_train", len(df_hlt_train))
# print("Length of Health_test", len(df_hlt_test))
# print("Length of ent_train", len(df_ent_train))
# print("Length of ent_test", len(df_ent_test))
file =open('/data/parush/wtwt/wtwt_extracted.json','r')
data = json.load(file)
df = pd.DataFrame(data)
df = df.drop(columns = 'tweet_id')
df = df.rename(columns={"merger": "target"})
df['domain'] = np.where(df['target']== 'FOXA_DIS', 'entertainment', 'healthcare')
df

,target,stance,tweet,domain
0,CI_ESRX,support,Cigna and ESI set to merge. Here we go...,healthcare
1,CI_ESRX,unrelated,Express Scripts Closes Acquisition Of eviCore;...,healthcare
2,CI_ESRX,comment,RT @_diginsurance: Cigna-Express Scripts deal ...,healthcare
3,CI_ESRX,support,Here's the just-released 400+ page merger prox...,healthcare
4,CI_ESRX,support,Cigna nears deal for Express Scripts https://t...,healthcare
...,...,...,...,...
44318,FOXA_DIS,comment,@Almightyvonzzus @Lampshadej Nah Disney bought...,entertainment
44319,FOXA_DIS,unrelated,what if the way Marvel/Disney brings in Fox’s ...,entertainment
44320,FOXA_DIS,comment,#Deadpool creator Rob Liefeld is not in favour...,entertainment
44321,FOXA_DIS,unrelated,"Hey @Disney, I could use a buyout.",entertainment


In [20]:
classes = {'support':0, 'refute': 1, 'comment': 2, 'unrelated': 3}


In [21]:
vectorizer = 'tfidf'   # set 'count' or 'tfidf'
analyzer = 'both'  # set 'word' or 'both' ( word and char)

In [22]:
if vectorizer == 'count':
    if analyzer == 'word':
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2,5))
else:
    if analyzer == 'word':
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(2,5))
        
        
        
        

In [23]:
def in_splitter(df, domain, classes):
    print("Started splitting {}, pre-processing".format(domain))
    df = df[df['domain'] == domain]
    X = df[['target','tweet', 'domain']]
    y = df[['stance']] 
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True,stratify = y)
    
    train_corpus = X_train_['tweet'].tolist()
    train_labels = [classes[i] for i in y_train_['stance'].tolist()]
    test_corpus = X_test_['tweet'].tolist()
    test_labels = [classes[i] for i in y_test_['stance'].tolist()]
    
    if analyzer == 'word':
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        return ngram_vectorized_data, train_labels, test_ngram_vectorized_data, test_labels
    else:
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        char_vectorized_data = char_vectorizer.fit_transform(train_corpus)
        l = np.hstack((ngram_vectorized_data.toarray(), char_vectorized_data.toarray()))
        train_vectorized_data = sparse.csr_matrix(l)
     
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        test_char_vectorized_data = char_vectorizer.transform(test_corpus)
        l2 = np.hstack((test_ngram_vectorized_data.toarray(), test_char_vectorized_data.toarray()))
        test_vectorized_data = sparse.csr_matrix(l2)
        
        return train_vectorized_data, train_labels, test_vectorized_data,test_labels

In [24]:
domains = {'ent': 'entertainment', 'hlt':'healthcare'}
X_train, y_train, X_test, y_test =  in_splitter(df, domains['hlt'],classes)

Started splitting healthcare, pre-processing


MemoryError: Unable to allocate 64.6 GiB for an array with shape (22101, 392049) and data type float64

In [8]:
unique,count = np.unique(y_train,return_counts=True)
dict(zip(unique, count))

{0: 3559, 1: 2598, 2: 8346, 3: 7598}

In [9]:
len(y_train)

22101

In [10]:
unique,count = np.unique(y_test,return_counts=True)
dict(zip(unique, count))

{0: 1187, 1: 866, 2: 2782, 3: 2532}

In [11]:
from sklearn.model_selection import StratifiedKFold

In [12]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']


for score in scores:
    
    print("# Tuning hyper-parameters for %s" % score)
    print()
    cv = StratifiedKFold(n_splits=2, shuffle=True, random_state = 2 )
    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score, cv = cv
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred, digits = 4,))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.094 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.094 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.847 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.094 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.823 (+/-0.008) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.847 (+/-0.001) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.803 (+/-0.001) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.823 (+/-0.008) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.806 (+/-0.001) for {'C': 1, 'kernel': 'linear'}
0.802 (+/-0.002) for {'C': 10, 'kernel': 'linear'}
0.802 (+/-0.002) for {'C': 100, 'kernel': 'linear'}
0.802 (+/-0.002) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:


              precision    recall  f1-score   support

           0     0.

In [13]:
print(classification_report(y_true, y_pred, digits = 4,))

              precision    recall  f1-score   support

           0     0.7755    0.7801    0.7778      1187
           1     0.7826    0.7691    0.7758       866
           2     0.7605    0.8102    0.7845      2782
           3     0.9059    0.8436    0.8736      2532

    accuracy                         0.8120      7367
   macro avg     0.8061    0.8007    0.8029      7367
weighted avg     0.8155    0.8120    0.8130      7367



# Test on other Target
False

In [50]:
X_test_, y_test_ = get_test_data_and_labels(test_data_file_m,TARGETS_m[2])

In [51]:
y_true_, y_pred_ = y_test_, clf.predict(X_test_)
print(classification_report(y_true_, y_pred_, digits = 4, labels = [0,1]))

              precision    recall  f1-score   support

           0     0.2414    0.4565    0.3158        46
           1     0.6622    0.5185    0.5816       189

   micro avg     0.5064    0.5064    0.5064       235
   macro avg     0.4518    0.4875    0.4487       235
weighted avg     0.5798    0.5064    0.5296       235

